In [1]:
import numpy as np
import tensorly as tl
import pandas as pd
import pickle

In [2]:
data_beatles = pd.read_csv('sbeatles.csv', delimiter=' , ', engine='python', names=['subject', 'predicate', 'object'], skiprows=1)
data_beatles = data_beatles.dropna()
data_beatles.head()

,subject,predicate,object
0,"""http://dbpedia.org/resource/The_Beatles""","""http://www.w3.org/1999/02/22-rdf-syntax-ns#type""","""http://dbpedia.org/class/yago/Quartets"" ,"
1,"""http://dbpedia.org/resource/The_Beatles""","""http://www.w3.org/1999/02/22-rdf-syntax-ns#type""","""http://sw.opencyc.org/2008/06/10/concept/Mx4r..."
2,"""http://dbpedia.org/resource/The_Beatles""","""http://www.w3.org/1999/02/22-rdf-syntax-ns#type""","""http://sw.opencyc.org/2008/06/10/concept/Mx4r..."
3,"""http://dbpedia.org/resource/The_Beatles""","""http://www.w3.org/1999/02/22-rdf-syntax-ns#type""","""http://dbpedia.org/ontology/MusicalArtist"" ,"
4,"""http://dbpedia.org/resource/The_Beatles""","""http://www.w3.org/1999/02/22-rdf-syntax-ns#type""","""http://dbpedia.org/class/yago/MusicalGroupsEs..."


In [3]:
data_beatles.describe()

,subject,predicate,object
count,4980,4980,4980
unique,1888,117,1388
top,"""http://dbpedia.org/resource/Eleanor_Rigby""","""http://dbpedia.org/property/wikilink""","""http://dbpedia.org/resource/The_Beatles"" ,"
freq,269,1719,512


### Load data

In [4]:
subjects = list(data_beatles['subject'].unique())
objects = list(data_beatles['object'].unique())
predicates = list(data_beatles['predicate'].unique())

k, l, m = len(subjects), len(objects), len(predicates)

### Create Tensor

### Run PARAFAC decomposition

In [5]:
with open('pp_parafac_ub_32_norm.pkl', 'rb') as f:
    D = pickle.load(f)

In [6]:
print(D.factors[0].shape)
print(D.factors[1].shape)
print(D.factors[2].shape)

(1888, 32)
(1388, 32)
(117, 32)


### Results

In [7]:
D.weights

array([54.775482 , 44.512054 , 14.933094 , 46.87411  , 26.968466 ,
       28.611624 , 30.647722 , 14.660114 , 16.07665  , 38.75045  ,
       20.840467 , 28.131765 , 24.282003 , 34.122986 , 36.721058 ,
       18.307045 , 14.850482 , 26.647234 , 11.118253 , 19.31693  ,
       26.728004 , 24.534616 , 22.560652 , 15.482001 , 26.450056 ,
       12.285444 , 28.17897  , 13.5755415, 14.107669 , 20.753212 ,
       25.13338  , 18.264532 ], dtype=float32)

In [8]:
def get_top_predicates(pfi, predicates):
    U_3 = np.absolute(D.factors[2][::,pfi])
    top_predicates = np.where(U_3 >= 0.1)[0]
    ans = [(p, predicates[p], U_3[p]) for p in top_predicates]
    
    return ans

In [9]:
def get_top_resources(pfi, top_predicates):
    U_1 = np.absolute(D.factors[0][::, pfi])
    U_2 = np.absolute(D.factors[1][::, pfi])
    
    subs = np.array(subjects)
    objs = np.array(objects)
    resources = list()
    ans = list()
    
    for i, _, U_3_i in top_predicates:
        A = np.dot(U_3_i, U_1)
        B = np.dot(U_3_i, U_2)
        
        Aarg = A.argsort()[::-1]
#         print('Subjects: ')
        for i in Aarg[:5]:
            if A[i] >= 0.0001 and (subs[i] not in resources):
                resources.append(subs[i])
                ans.append((subs[i], A[i]))
#                 print((subs[i], A[i]))
        
        Barg = B.argsort()[::-1]
        
#         print('Objects: ')
        for i in Barg[:5]:
            if B[i] >= 0.0001 and (objs[i] not in resources):
                resources.append(objs[i])
                ans.append((objs[i], B[i]))
#                 print((objs[i], B[i]))
        
    sans = np.array(sorted(ans, key=lambda x: x[1])[::-1])
    print(sans)

### Group pfn

In [10]:
# 8
pfn = 24
pf0 = get_top_predicates(pfn, predicates)
print(pf0)
get_top_resources(pfn, pf0)

[(53, '"http://dbpedia.org/ontology/writer"', 0.7301684), (54, '"http://dbpedia.org/property/writer"', 0.6568003), (86, '"http://dbpedia.org/ontology/composer"', 0.1259725), (87, '"http://dbpedia.org/property/composer"', 0.1259725)]
[['"http://dbpedia.org/resource/Lennon/McCartney" ,' '0.72913355']
 ['"http://dbpedia.org/resource/You%27ve_Got_to_Hide_Your_Love_Away"'
  '0.2868187']
 ['"http://dbpedia.org/resource/Little_Child"' '0.22933038']
 ['"http://dbpedia.org/resource/I%27m_Looking_Through_You"' '0.22931483']
 ['"http://dbpedia.org/resource/I%27ll_Get_You"' '0.2292629']
 ['"http://dbpedia.org/resource/If_You%27ve_Got_Trouble"' '0.22924736']
 ['"http://dbpedia.org/resource/Lennon-McCartney" ,' '0.037581086']
 ['"http://dbpedia.org/resource/Allmusic" ,' '0.0017926617']
 ['"http://dbpedia.org/resource/Abbey_Road_Studios" ,' '0.0011203188']
 ['"http://dbpedia.org/resource/John_Lennon" ,' '0.0010605354']]
